In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(0.25)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(0.25)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(0.25)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.25)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Telangana_bus_details.csv', index=False)

driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/hyderabad-to-khammam: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=129.0.6668.90); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6AE47B095+29557]
	(No symbol) [0x00007FF6AE3EFA50]
	(No symbol) [0x00007FF6AE2AB56A]
	(No symbol) [0x00007FF6AE2B213E]
	(No symbol) [0x00007FF6AE2B44B7]
	(No symbol) [0x00007FF6AE2B4570]
	(No symbol) [0x00007FF6AE2F942B]
	(No symbol) [0x00007FF6AE32718A]
	(No symbol) [0x00007FF6AE2F2E86]
	(No symbol) [0x00007FF6AE3273A0]
	(No symbol) [0x00007FF6AE34851C]
	(No symbol) [0x00007FF6AE326F33]
	(No symbol) [0x00007FF6AE2F116F]
	(No symbol) [0x00007FF6AE2F22D1]
	GetHandleVerifier [0x00007FF6AE7AC96D+3378253]
	GetHandleVerifier [0x00007FF6AE7F8497+3688311]
	GetHandleVerifier [0x00007FF6

NameError: name 'driver' is not defined

In [4]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),23:10,06h 25m,05:35,4.5,829,14 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),23:50,05h 45m,05:35,4.4,INR 929,27 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Zingbus Plus,A/C Seater / Sleeper (2+1),23:38,06h 27m,06:05,4.5,983,22 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdSitara Travels,NON A/C Push Back (2+2),22:00,06h 00m,04:00,4.6,INR 944,9 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,Scania AC Multi Axle Sleeper (2+1),23:59,06h 31m,06:30,4.5,INR 1469,11 Seats available
...,...,...,...,...,...,...,...,...,...,...
200,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Laxmi Sravanthi Siva Tours And Travels,Non A/C Seater / Sleeper (2+1),21:50,07h 05m,04:55,3.7,INR 899,14 Seats available
201,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Orange Tours And Travels,Volvo Multi-Axle Sleeper A/C (2+1),23:45,06h 30m,06:15,3.5,INR 1600,25 Seats available
202,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,V Kaveri Travels,A/C Sleeper (2+1),22:00,07h 45m,05:45,4.6,INR 1420,18 Seats available
203,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,SAI PRASANNA TRAVELS,Non A/C Seater / Sleeper (2+1),21:40,07h 10m,04:50,3.8,INR 1390,1 Seat available
